## QHyper usecase

### Create instance of problem

In [20]:
from QHyper.problems.knapsack import KnapsackProblem
from QHyper.problems.tsp import TSPProblem

PROBLEM_TYPE = 'knapsack'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one
 
if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
    # and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
    print(problem.knapsack.items)
    
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(
        number_of_cities=3,
    )
    params_cofing = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [21]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: -2*x0 - 2*x1 - x2
Constraints:
    -x3 - x4 + 1
    -x0 - x1 - x2 + x3 + 2*x4


### Use VQA to solve knapsack problem

In [22]:
from QHyper.solvers.vqa.base import VQA

In [23]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function

tester_config = {
    "pqc": {
        "type": "wfqaoa",
        "layers": 5,
    }
}

tester = VQA(problem, config=tester_config)

In [24]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
    },
    "pqc": {
        "type": "hqaoa",
        "layers": 5,
    },
}
vqa = VQA(problem, config=solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, None, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [25]:
# Run VQA with provided initial parameters
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

Best params: {'angles': array([[1.64230366, 0.08750879, 0.        , 0.12854769, 2.49365561],
       [0.93907822, 0.62299303, 1.10213747, 0.41506598, 1.45096723]]), 'hyper_args': array([1. , 2.5, 2.5])}


In [26]:
# Here created ealier tester will be used to evaluate results

best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -1.7860315362678931
Params used for optimizer:
[[1.64230366 0.08750879 0.         0.12854769 2.49365561]
 [0.93907822 0.62299303 1.10213747 0.41506598 1.45096723]],
and params used for hyper optimizer: [1.  2.5 2.5]


#### Using hyper optimizers

In [48]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

from QHyper.optimizers.random import Random

solver_config = {
    'pqc': {
        'type': 'hqaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    }
}

vqa = VQA(problem, config=solver_config)

random = Random(processes=5, number_of_samples=100, bounds=hyper_optimizer_bounds)
best_params_h = vqa.solve(params_cofing, random)

100%|██████████| 100/100 [29:33<00:00, 17.73s/it]


In [49]:
# Again, final results will be evaluated with tester

best_results = tester.evaluate(best_params_h)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params_h['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -0.1508898084256115
Params used for optimizer:
[0.62729292 0.40374954 0.40249242 0.54078253 0.51545703 0.97628969
 1.04418123 0.92627657 0.95441227 1.01531624],
and params used for hyper optimizer: [1.02005268 2.10821942 1.94148846 2.14637279 1.88565744]
